#**Preprocessing of US Datasets**

In [ ]:
from google.colab import drive
import pandas as pd
import math
import os
prefix = '/content/drive'
drive.mount(prefix, force_remount=True)

##**Walkability Data**

###Obtain data

In [ ]:
df_walk = pd.read_csv('/content/drive/My Drive/GoSQL_Project_Deliverables/Datasets/EPA_SmartLocationDatabase_V3_Jan_2021.csv')
df_walk.head()

###Preprocess the data

Later we will need a column 'CensusTract' to join with another dataset to determine zipcodes. Let's determine the 11 digit census tract number per the definition here:
https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

In [ ]:
df_walk['CensusTract'] = (df_walk['STATEFP']*(10**9) + df_walk['COUNTYFP']*(10**6) + df_walk['TRACTCE'])
df_walk.head()

In [ ]:
# There are 117 columns in the dataset...let's narrow that down to ones we are interested in
columns_to_keep = ['CensusTract', 'NatWalkInd', 'TotPop', 'Ac_Total', 'Ac_Water', 'Ac_Land', 'Ac_Unpr', 'Workers', 'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk']
df_walk = df_walk[columns_to_keep]
df_walk.head()

The main purporse of this dataset is to obtain a walkability index per zipcode. We will not accept tuples where 'NatWalkInd' is missing, however we will accept other columns with missing data as the EPA was able to calculate a walkability index for this location. Let's clean the data by removing any tuples where 'NatWalkInd' is missing or contains an illegal value (any value not between 1 and 20).

In [ ]:
df_walk['NatWalkInd'] = pd.to_numeric(df_walk['NatWalkInd'], errors='coerce') #convert natwalkInd to a float
df_walk_clean = df_walk[((df_walk['NatWalkInd'].notna()) & (df_walk['NatWalkInd'] >= 1) & (df_walk['NatWalkInd'] <= 20))]

Ultimately our project aims to compare areas by zipcode, but currently df_walk associates collected data by census block group. Before further processing the EPA dataset, let's pull our zip tract data and join on Census tract to obtain a zipcode for each tuple.

In [ ]:
df_zip = pd.read_csv('/content/drive/My Drive/GoSQL_Project_Deliverables/Datasets/zip_tract.csv')
# Ensure there are no duplicates
print(df_zip.shape)
df_zip = df_zip.drop_duplicates(subset=['CensusTract'])
print(df_zip.shape)
df_zip.head()

In [ ]:
df_walk_zips = pd.merge(df_walk_clean, df_zip, on='CensusTract', how='left')
# Number of rows should stay the same, but number of columns should increase
print(df_walk_zips.shape)
print(df_walk_clean.shape)

Check to see if any zip code values were not found

In [ ]:
df_zip_not_found = df_walk_zips[(df_walk_zips['Zip'].isna())]
df_zip_not_found.shape

Check if any zip codes are 6 digits or more. All should be 5.

In [ ]:
zip_too_big = df_walk_zips[df_walk_zips['Zip'] > 99999]
zip_too_big.shape # should have 0 rows

44,921 census tracts could not be aligned with a zip code. Drop all erroneous values and keep the remaining valid tuples (220,740 - 44,921 valid tuples)

In [ ]:
df_walk_zips = df_walk_zips[(df_walk_zips['Zip'].notna())]
df_walk_zips.shape # should have 220,740 - 44,921 rows

Aggregate data per zipcode. Most columns will require summation. NatWlkInd is an exception, we will take the average to get a walkability index representative of the zip code.

In [ ]:
df_walk_zips.columns

In [ ]:
df_walk_zips = df_walk_zips[['Zip', 'NatWalkInd', 'TotPop', 'Ac_Total', 'Ac_Water',
       'Ac_Land', 'Ac_Unpr', 'Workers', 'R_LowWageWk', 'R_MedWageWk',
       'R_HiWageWk']] # drop unneeded columns
aggregated_walk_zips = df_walk_zips.groupby('Zip').agg({
    'NatWalkInd': 'mean',
    'TotPop': 'sum',
    'Ac_Total': 'sum',
    'Ac_Water': 'sum',
    'Ac_Land': 'sum',
    'Ac_Unpr': 'sum',
    'Workers': 'sum',
    'R_LowWageWk': 'sum',
    'R_MedWageWk': 'sum',
    'R_HiWageWk': 'sum'
}).reset_index()
aggregated_walk_zips.head()

In [ ]:
# Create a column representing the % of land protected from development (e.g. park, natural area or conservation area)
aggregated_walk_zips['Ac_Prot'] = (aggregated_walk_zips['Ac_Land'] - aggregated_walk_zips['Ac_Unpr'])



Rename columns to use camel casing and match rest of database vars

In [ ]:
aggregated_walk_zips.rename(columns={'Ac_Total': 'AreaTotal',
                                     'Ac_Land': 'AreaLand',
                                     'Ac_Unpr': 'AreaUnpr',
                                     'Ac_Water': 'AreaWater',
                                     'Ac_Prot': 'AreaProt',
                                     'Workers': 'CntWorkers',
                                     'R_LowWageWk': 'CntLowWage',
                                     'R_MedWageWk': 'CntMedWage',
                                     'R_HiWageWk': 'CntHiWage',
                                     }, inplace=True)
aggregated_walk_zips.columns

Prior to exporting let's convert zip to a 5 digit string, ensure null values are assigned default values, and negative numbers are assigned default values. No need to check NatWalkInd as we already dropped all invalid entries. No need to check Zip as we already removed all invalid entries.

In [ ]:
aggregated_walk_zips['Zip'] = aggregated_walk_zips['Zip'].astype(int).astype(str).str.zfill(5) ## change zip to a string and pad with zeros if necessary
columns_to_mod = ['TotPop', 'AreaTotal', 'AreaWater', 'AreaLand',
       'AreaUnpr', 'CntWorkers', 'CntLowWage', 'CntMedWage', 'CntHiWage',
       'AreaProt']
aggregated_walk_zips[columns_to_mod] = aggregated_walk_zips[columns_to_mod].fillna(0)
aggregated_walk_zips[columns_to_mod] = aggregated_walk_zips[columns_to_mod].applymap(lambda x: 0 if x < 0 else x)
aggregated_walk_zips.head()

Export data

In [ ]:
aggregated_walk_zips.to_csv('/content/drive/My Drive/GoSQL_Project_Deliverables/Datasets/us_walkability_cleaned.csv', index=False)

##**Life Expectancy Dataset**

In [ ]:
# Load the CSV files into DataFrames
life_expectancy_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/us_life_expectancy.csv', encoding='ISO-8859-1', dtype={'Census_Tract': str})
fips_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/fips-by-state.csv', encoding='ISO-8859-1', dtype={'fips': str})
life_expectancy_df.head()

In [ ]:
fips_df.head()

In [ ]:
# Ensure the Census Tract columns are in a consistent format for joining
life_expectancy_df['State'] = life_expectancy_df['State'].astype(str).str.strip()
life_expectancy_df['County'] = life_expectancy_df['County'].astype(str).str.strip()
fips_df['state'] = fips_df['state'].astype(str).str.strip()
fips_df['county'] = fips_df['county'].astype(str).str.strip()

In [ ]:
# Merge the DataFrames on the State and County columns
merged_df = pd.merge(life_expectancy_df, fips_df, how='inner', left_on=['State', 'County'], right_on=['state', 'county'])
merged_df.head()

In [ ]:
# Select the required columns
final_df = merged_df[['fips', 'Census_Tract', 'Life Expectancy', 'Life Expectancy Range', 'Life Expectancy Standard Error', 'County', 'State']].copy()

# Concatenate the fips and Census_Tract values to create a new column called CensusTract
final_df.loc[:, 'CensusTract'] = final_df['fips'].astype(str) + final_df['Census_Tract'].astype(str)

In [ ]:
# Drop the fips and Census_Tract columns
final_df = final_df.drop(columns=['fips', 'Census_Tract'])

# Filter the DataFrame to exclude rows where the state is 'ty'
final_df = final_df[final_df['State'] != 'ty']

In [ ]:
# Save the resulting DataFrame to a new CSV file
final_df.to_csv('/content/drive/MyDrive/CIS550/project/us/us_life_expectancy_cleaned.csv', index=False)

In [ ]:
zip_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/zip_tract.csv', dtype={'CensusTract': str})
zip_df.head()

In [ ]:
cleaned_life_expectancy_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/us_life_expectancy_cleaned.csv', dtype={'CensusTract': str})
cleaned_life_expectancy_df.head()

In [ ]:
# prompt: i need all the CensusTract in zip_df that are not in cleaned_life_expectancy_df. Deduplicate the CensusTracts. Then put these missing CensusTracts in a separate csv file

missing_tracts = set(zip_df['CensusTract']) - set(cleaned_life_expectancy_df['CensusTract'])
missing_tracts = list(missing_tracts)

# Deduplicate the CensusTracts
missing_tracts_df = pd.DataFrame({'CensusTract': missing_tracts})
missing_tracts_df = missing_tracts_df.drop_duplicates()

# Save the missing CensusTracts to a new CSV file
missing_tracts_df.to_csv('/content/drive/MyDrive/CIS550/project/us/missing_tracts.csv', index=False)


In [ ]:
# prompt: please merge missing_tracts_df with cleaned_life_expectancy_df making sure that it is a left outer join with missing_tracts_df on the left. Merge on the CensusTract field which is a string

all_tracts_life_expectancy_df = pd.merge(missing_tracts_df, cleaned_life_expectancy_df, how='outer', left_on='CensusTract', right_on='CensusTract')
all_tracts_life_expectancy_df.head()

In [ ]:
all_tracts_life_expectancy_df.to_csv('/content/drive/MyDrive/CIS550/project/us/all_tracts_life_expectancy.csv', index=False)

In [ ]:
all_life_expectancy_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/all_tracts_life_expectancy.csv', dtype={'CensusTract': str})
all_life_expectancy_df.head()

In [ ]:
all_zips_df = pd.read_csv('/content/drive/MyDrive/CIS550/project/us/zip_tract.csv', dtype={'CensusTract': str})
all_zips_df.head()

In [ ]:
# prompt: Are there any values in CensusTract in all_zips_df that do not appear in the CensusTract column of all_life_expectancy_df. If so how many?

missing_tracts = set(all_zips_df['CensusTract']) - set(all_life_expectancy_df['CensusTract'])
num_missing_tracts = len(missing_tracts)

print(f"There are {num_missing_tracts} values in CensusTract in all_zips_df that do not appear in the CensusTract column of all_life_expectancy_df.")


##**Income Dataset**

##**Housing Dataset**

In [ ]:
rent_fp = '/content/drive/MyDrive/CIS550/project/us/us_avg_rent_prices.csv'
rent_df = pd.read_csv(rent_fp, dtype={'Zip': str})
rent_df.head()

In [ ]:
house_price_fp = '/content/drive/MyDrive/CIS550/project/us/us_avg_house_prices.csv'
house_price_df = pd.read_csv(house_price_fp, dtype={'Zip': str})
house_price_df.head()

In [ ]:
# prompt: For both house_price_df and rent_df if any value in the ZIP column isn't five characters long then it needs to be padded with zeros until it is five characters long. Also for the AvgPrice and AvgRent columns the values should only be to two decimal places. Remember that house_price_df has a AvgPrice column but not a AvgRent column and rent_df has a AvgRent column but not a AvgPrice column

for df in [house_price_df, rent_df]:
  df['Zip'] = df['Zip'].str.zfill(5)
  if 'AvgPrice' in df.columns:
    df['AvgPrice'] = df['AvgPrice'].apply(lambda x: round(x, 2))
  if 'AvgRent' in df.columns:
    df['AvgRent'] = df['AvgRent'].apply(lambda x: round(x, 2))


In [ ]:
rent_df.head()

In [ ]:
house_price_df.head()

In [ ]:
# prompt: save the house_price_df as a new csv file and save the rent_df as a new csv file

house_price_df.to_csv('/content/drive/MyDrive/CIS550/project/us/us_avg_house_prices_cleaned.csv', index=False)
rent_df.to_csv('/content/drive/MyDrive/CIS550/project/us/us_avg_rent_prices_cleaned.csv', index=False)


##**Air Quality Dataset**